# 学习 scipy 积分/微分方程 解法 —— scipy.integration

https://docs.scipy.org/doc/scipy-1.5.4/reference/tutorial/integrate.html

## Ordinary differential equations (solve_ivp)

- The function solve_ivp is available in SciPy for integrating a first-order vector differential equation

solve_ivp 可用于求解一阶矢量微分方程：

<img src="./img/A01-Yt.jpg"></img>

**y**是一个 N 维矢量，**f** 是一个 R^N 到 R^N 的映射。

- A higher-order ordinary differential equation can always be reduced to a differential equation of this type by introducing intermediate derivatives into the **y** vector.

高阶常微分方程，通过引入中间变量，总可以转为 **y** 矢量。对吗？下面举个例子（w 是 z 的函数）

w'' - zw = 0

初始条件为

<img src="./img/A02-w0.jpg"></img>

在此初始条件下， w 是著名的 Airy 函数（艾里函数）

做如下转换

<img src="./img/A03-Yww.jpg"></img>

就可以转为一阶微分方程（注意没必要写成矩阵相乘形式，不要误会了）

<img src="./img/A04-dyy.jpg"></img>

下面写代码实现

In [8]:
from scipy.integrate import solve_ivp
from scipy.special import gamma, airy
y1_0 = +1 / 3**(2/3) / gamma(2/3)
y0_0 = -1 / 3**(1/3) / gamma(1/3)

# 初值
y0 = [y0_0, y1_0]

# f 函数
def func(t, y):
    return [t*y[1],y[0]]

# t 初值和终值
t_span = [0, 4]

# 求解
sol1 = solve_ivp(func, t_span, y0)

# 查看 t
print(f"sol1.t: {sol1.t}")

sol1.t: [0.         0.10097672 1.04643602 1.91060117 2.49872472 3.08684827
 3.62692846 4.        ]


In [6]:
# 查看  t 对应的 y[1] 就是 w(t)
print(f"sol1.y[1]: {sol1.y[1]}")

sol1.y[1]: [0.35502805 0.328952   0.12801343 0.04008508 0.01601291 0.00623879
 0.00356316 0.00405982]


In [9]:
# 查看精确解
print("airy(sol.t)[0]:  {}".format(airy(sol1.t)[0]))

airy(sol.t)[0]:  [0.35502805 0.328952   0.12804768 0.03995804 0.01575943 0.00562799
 0.00201689 0.00095156]


可见误差有点大，引入相对误差和绝对误差

In [10]:
sol2 = solve_ivp(func, t_span, y0, rtol=1e-8, atol=1e-8)
print("sol2.y[1][::6]: {}".format(sol2.y[1][0::6]))
print("airy(sol2.t)[0][::6]: {}".format(airy(sol2.t)[0][::6]))

sol2.y[1][::6]: [0.35502805 0.19145234 0.06368989 0.0205917  0.00554734 0.00106409]
airy(sol2.t)[0][::6]: [0.35502805 0.19145234 0.06368989 0.0205917  0.00554733 0.00106406]


返回感兴趣的 t 值

In [12]:
import numpy as np
t = np.linspace(0, 4, 11)
sol3 = solve_ivp(func, t_span, y0, t_eval=t)

print(f"t={sol3.t}")
print(f"w={sol3.y[1]}")

t=[0.  0.4 0.8 1.2 1.6 2.  2.4 2.8 3.2 3.6 4. ]
w=[0.35502805 0.25490008 0.16989792 0.10609285 0.06253061 0.03505822
 0.01877444 0.00978726 0.0052999  0.0035991  0.00405982]
